## Описание проекта

Нам предоставлены данные  добывающей компании. Нужно решить, где бурить новую скважину.

Шаги для выбора локации обычно такие:

     В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
     Строят модель для предсказания объёма запасов в новых скважинах;
     Выбирают скважины с самыми высокими оценками значений;
     Определяют регион с максимальной суммарной прибылью отобранных скважин.
     
Нам предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Необходимо построить модель для определения региона, где добыча принесёт наибольшую прибыль. Нужно Проанализировать возможную прибыль и риски техникой Bootstrap.

#### Данные

id — уникальный идентификатор скважины;

f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);

Целевой признак:
    
product — объём запасов в скважине (тыс. баррелей).

### Изучение данных

Импортируем библиотеки

In [20]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy import stats as st

Загружаем данные:

In [2]:
import os

pth1 = 'geo_data_0.csv'
pth2 = '/datasets/geo_data_0.csv'

pth3 = 'geo_data_1.csv'
prh4 = '/datasets/geo_data_1.csv'

pth5 = 'geo_data_2.csv'
pth6 = '/datasets/geo_data_2.csv'

if os.path.exists(pth1):
    df_1 = pd.read_csv(pth1)
    df_2 = pd.read_csv(pth3)
    df_3 = pd.read_csv(pth5)
elif os.path.exists(pth2):
    df_1 = pd.read_csv(pth2)
    df_2 = pd.read_csv(pth4)
    df_3 = pd.read_csv(pth6)
else:
    print('Something is wrong')

Изучаем датафреймы:

In [3]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Данные в полном порядке. Пропусков нет. Можно приступать к обуению модели

### Обучение модели

Создаем функцию для обуения модели Линейной регрессии. 

Выделяем целевой признак - 'product'.

Делим выборку на обучающую и валидную в пропорции 75:25

Обучаем модель и делаем предсказания на валидационной выборке.

Сохраняем предсказания и правильные ответы на валидационной выборке для каждого региона.

In [22]:
def line_model(df):
    target = df['product']
    features = df.drop(['id','product'], axis=1)

    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)
  
    model = LinearRegression()
    model.fit(features_train, target_train) 
    predict = model.predict(features_valid) 
    predict = pd.Series(predict)
    return target_valid, predict

target1,predict1 = line_model(df_1)
target2,predict2 = line_model(df_2)
target3,predict3 = line_model(df_3)

print(target1,predict1)
print(target2,predict2)
print(target3,predict3)

71751     10.038645
80493    114.551489
2655     132.603635
53233    169.072125
91141    122.325180
            ...    
12581    170.116726
18456     93.632175
73035    127.352259
63834     99.782700
43558    177.821022
Name: product, Length: 25000, dtype: float64 0         95.894952
1         77.572583
2         77.892640
3         90.175134
4         70.510088
            ...    
24995    103.037104
24996     85.403255
24997     61.509833
24998    118.180397
24999    118.169392
Length: 25000, dtype: float64
71751     80.859783
80493     53.906522
2655      30.132364
53233     53.906522
91141      0.000000
            ...    
12581    137.945408
18456    110.992147
73035    137.945408
63834     84.038886
43558     53.906522
Name: product, Length: 25000, dtype: float64 0         82.663314
1         54.431786
2         29.748760
3         53.552133
4          1.243856
            ...    
24995    136.869211
24996    110.693465
24997    137.879341
24998     83.761966
24999     53.958466


Модель работает

Рассчитываем и выводим  на экран средний запас предсказанного сырья и RMSE модели.

In [38]:
predict_mean_crude1 = predict1.mean()
rmse1 = mean_squared_error(target1, predict1)**0.5
print('средний запас предсказанного сырья 1-го региона', round(predict_mean_crude1, 2))
print('RMSE 1-го региона =', round(rmse1, 3))

средний запас предсказанного сырья 1-го региона 92.59
RMSE 1-го региона = 37.579


In [39]:
predict_mean_crude2 = predict2.mean()
rmse2 = mean_squared_error(target2, predict2)**0.5
print('средний запас предсказанного сырья 2-го региона', round(predict_mean_crude2, 2))
print('RMSE 2-го региона =', round(rmse2, 3))

средний запас предсказанного сырья 2-го региона 68.73
RMSE 2-го региона = 0.893


In [41]:
predict_mean_crude3 = predict3.mean()
rmse3 = mean_squared_error(target3, predict3)**0.5
print('средний запас предсказанного сырья 3-го региона', round(predict_mean_crude3, 2))
print('RMSE 3-го региона =', round(rmse3, 3))

средний запас предсказанного сырья 3-го региона 94.97
RMSE 3-го региона = 40.03


Имортировали библиотеки, изучили датафреймы.

Создали функцию для обучения модели Линейной регрессии

Обучили модель и сохранили предсказания на валидной выборке

### Подготовка к расчёту прибыли:

In [23]:
budget = 10**10
barell_profit = 450000
wells=500
numder_of_best_wells = 200

Рассчитываем достаточный объём сырья для безубыточной разработки новой скважины. 

In [11]:
sufficient_volume = budget / (barell_profit*numder_of_best_wells)
print(round(sufficient_volume,2))

111.11


Достаточный объём сырья для безубыточной разработки новой скважины равен 111,11 тыс баррелей.

Среднее значение в исследуемых регионах:
   1) В регионе 1 - 92.59 тыс. баррелей
   
   2) В регионе 2 - 68.73 тыс баррелей
   
   3) В регионе 3 - 94.97 тыс баррелей
   
Ни в каждом регионе среднее значение не выше достаточного объёма сырья для безубыточной разработки новой скважины

### Pасчёт прибыли по выбранным скважинам и предсказаниям модели:

Создаем функцию для рассчета прибыли по выбранным скважинам и предсказаниям модели.

Выбираем лучше 200 скважин

Возвращаем значение средней прибыли по скважинам

In [12]:
def predicted_profit (target, predict):
    sort_predict = predict.reset_index(drop=True).sort_values(ascending=False)
    best_target = target.reset_index(drop=True)[sort_predict.index][:numder_of_best_wells]
    
    profit = (best_target.sum() * barell_profit - budget) / 1000000000
    return profit

Применяем технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли, для этого создаем функцию.

Найдим среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.

In [34]:
def Bootstrap(target, predict):
    alpha=0.025
    state = np.random.RandomState(12345)
    values = []
    for i in range(1000):
        target_subsample = target.reset_index(drop=True).sample(n=wells, replace=True, random_state=state)
        probs_subsample = predict[target_subsample.index]
        result = predicted_profit(target_subsample, probs_subsample)
        values.append(result)
  
    values = pd.Series(values)
    lower = values.quantile(0.025)

    mean = values.mean()
    confidence_interval = (lower, values.quantile(1-alpha))
    
    print('Средняя прибыль {:.2f} млдр'.format(mean))
    print('2.5% квантиль {:.2f}'.format(lower))
    print('Доверительный интервал:',  confidence_interval)
    print('Риск убытка:{}'.format((values<0).mean()))
    

Расчет прибыли по 1-му региону

In [35]:
Bootstrap(target1, predict1)

Средняя прибыль 0.40 млдр
2.5% квантиль -0.11
Доверительный интервал: (-0.11121554589049526, 0.9097669415534225)
Риск убытка:0.069


Расчет прибыли по 2-му региону

In [36]:
Bootstrap(target2, predict2)

Средняя прибыль 0.46 млдр
2.5% квантиль 0.03
Доверительный интервал: (0.03382050939898362, 0.852289453866036)
Риск убытка:0.015


Расчет прибыли по 3-му региону

In [37]:
Bootstrap(target3, predict3)

Средняя прибыль 0.40 млдр
2.5% квантиль -0.16
Доверительный интервал: (-0.16335041339560108, 0.9503595749237995)
Риск убытка:0.076


Исходя из результато расчета лучше всего себя показал 2-й регион. у него лучшие показали:
  - Средняя прибыль 0.46 млдр
  - 2.5% квантиль 0.03
  - Доверительный интервал: (0.03382050939898362, 0.852289453866036)
  - Риск убытка:0.015

Соответственно во 2-й регион  необходимо направть бюджет на разработку скважин.

## Общий вывод

Нам были предоставлены данные добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Шаги для выбора локации были следующими:

    В избранном регионе были собираны характеристики для скважин: качество нефти и объём её запасов;
    
    Построена модель для предсказания объёма запасов в новых скважинах;
    
    Выбираны скважины с самыми высокими оценками значений;
    
    Определен регион с максимальной суммарной прибылью отобранных скважин.
    

Были загружены, изуены  и подготовлены данные.  Выделен целевой признак.

Разбиты данные на обучающую и валидационную выборки в соотношении 75:25. Обучена модель и сделаны предсказания на валидационной выборке.

Вывели  на экран средний запас предсказанного сырья и RMSE модели по каждому региону:

   1) средний запас предсказанного сырья 1-го региона 92.59
      RMSE 1-го региона = 37.579
      
   2) средний запас предсказанного сырья 2-го региона 68.73
      RMSE 2-го региона = 0.893
   3) средний запас предсказанного сырья 3-го региона 94.97
      RMSE 3-го региона = 40.03   

Все ключевые значения для расчётов сохранили в отдельных переменных.

Рассчитали достаточный объём сырья для безубыточной разработки новой скважины. Сравнили полученный объём сырья со средним запасом в каждом регионе:

      Достаточный объём сырья для безубыточной разработки новой скважины равен 111,11 тыс баррелей.

      Среднее значение в исследуемых регионах:
      
            1) В регионе 1 - 92.59 тыс. баррелей
   
            2) В регионе 2 - 68.73 тыс баррелей
   
            3) В регионе 3 - 94.97 тыс баррелей
   
      Ни в каждом регионе среднее значение не выше достаточного объёма сырья для безубыточной разработки новой скважины.
      
В расчёте прибыли по выбранным скважинам и предсказаниям модели:

    Создаели функцию для рассчета прибыли по выбранным скважинам и предсказаниям модели.

    Выбирали лучше 200 скважин

    Вернули значение средней прибыли по скважинам      
      

Применяя технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли, для этого создаем функцию.

Нашли среднюю прибыль, 95%-й доверительный интервал и риск убытков:

    1-й регион:
             Средняя прибыль 0.40 млдр
             2.5% квантиль -0.11
             Доверительный интервал: (-0.11121554589049526, 0.9097669415534225)
             Риск убытка:0.069
             
    2-й регион:
             Средняя прибыль 0.46 млдр
             2.5% квантиль 0.03
             Доверительный интервал: (0.03382050939898362, 0.852289453866036)
             Риск убытка:0.015

    3-й регион:
             Средняя прибыль 0.40 млдр
             2.5% квантиль -0.16
             Доверительный интервал: (-0.16335041339560108, 0.9503595749237995)
             Риск убытка:0.076
             
Исходя из результато расчета лучше всего себя показал 2-й регион. у него лучшие показали.  Соответственно во 2-й регион необходимо направть бюджет на разработку скважин.          